##### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 
* 콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [8]:
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF 파일 경로 설정
pdf_filepath = '../data/콘텐츠분쟁해결_사례.pdf'

# 파일 존재 여부 확인 (파일이 없으면 오류 발생)
if not os.path.exists(pdf_filepath):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")

In [9]:
try:
    # 1. PDF 파일 로드
    loader = PyPDFLoader(pdf_filepath)  # PDF 파일을 로드할 객체 생성
    docs = loader.load()  # 문서를 전체 로드

    # 총 문서 개수 출력
    print(f"총 {len(docs)}개의 문서가 로드 되었습니다.", type(docs[0]))

    #  첫 번째 문서의 메타데이터 출력
    print("첫 번째 문서 메타데이터:")
    print(json.dumps(docs[0].metadata, indent=2, ensure_ascii=False))

    # 특정 인덱스(10번째) 문서의 내용 확인 (존재할 경우)
    if len(docs) > 10:
        print("\n10번째 문서 내용:", type(docs[10]))
        print(docs[10])  # 10번째 문서 출력

    #  2. 텍스트 분할 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, 
                                                   chunk_overlap=300,
                                                    separators=[
                                                    "\n【사건개요】",     #  법률 문서 섹션 구분자
                                                    "\n【쟁점사항】",     #  쟁점 부분 구분
                                                    "\n【처리경위】",     #  처리 과정 구분
                                                    "\n【처리결과】",     #  결과 부분 구분
                                                    "\n■", "\n\n", "\n", ".", " ", ""
                                                    ])
    split_docs = loader.load_and_split(text_splitter=text_splitter)  # 분할된 문서 로드

    # 분할된 문서 개수 출력
    print(f"\n분할된 문서의 개수: {len(split_docs)} 타입: {type(split_docs)}")

    # 10번째 분할된 문서 내용 출력 (존재할 경우)
    if len(split_docs) > 10:
        print("\n10번째 분할된 문서:")
        print(split_docs[10])

    # 3. Lazy Load 방식으로 문서 로드
    print("\nLazy Load 방식으로 문서 로드:")
    for i, doc in enumerate(loader.lazy_load()):
        if i < 5:  # 너무 많은 출력 방지 (예제: 처음 5개만 출력)
            print(json.dumps(doc.metadata, indent=2, ensure_ascii=False))

except Exception as e:
    # 오류 발생 시 메시지 출력
    print(f"오류 발생: {e}")

총 109개의 문서가 로드 되었습니다. <class 'langchain_core.documents.base.Document'>
첫 번째 문서 메타데이터:
{
  "producer": "Hancom PDF 1.3.0.410",
  "creator": "Hancom PDF 1.3.0.410",
  "creationdate": "2011-01-20T18:01:33+09:00",
  "title": "제 2절 영국사례",
  "moddate": "2011-01-20T18:01:33+09:00",
  "pdfversion": "1.4",
  "source": "../data/콘텐츠분쟁해결_사례.pdf",
  "total_pages": 109,
  "page": 0,
  "page_label": "1"
}

10번째 문서 내용: <class 'langchain_core.documents.base.Document'>
page_content='1. 게임
5
■ 1-가-1) 한국소비자원 조정전 상담사례2007_인터넷게임서비스 아이템 복구 요구【사건개요】신청인은 피신청인의 인터넷게임서비스를 이용자이다. 2007. 11. 14 피신청인은 신청인이 타 이용자에게서 11. 10. 구매한 아이템이 해킹 아이템이라며 동 아이템을 회수하고 신청인이 지급하였던 52,000,000딜(게임머니)을 환급하였다. 이에 신청인은 정상적인 경로로 구매한 아이템이므로 피신청인에게 아이템의 복구를 요구하였으나 이를 거절당하자. 임의로 회수된 인터넷게임서비스 아이템의 복구를 요구하였다.【쟁점사항】피신청인의 아이템 회수 행위가 정당한 지 여부【처리경위】피신청인의 이용약관 제11조(이용자의 의무) 제4항에는 이용자는 게임내에서나 웹사이트, 기타 개발사에서 제공한 모든 서비스 관련 내용은 저작권이 제작사에 있으므로, 이것을 제3자에게 제공하거나 판매하는 행위를 하여서는 안 되며 만약 이를 어길 시 법적인 제재를 받을 수 있다고 규정하고 있다.또한 동 아이템은 타 이용자가 해킹으로 분실한 것으로, 비록 신청인이 

In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader(pdf_filepath)  # PDF 파일을 로드할 객체 생성
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, 
                                                   chunk_overlap=300,
                                                    separators=[
                                                    "\n【사건개요】",     #  법률 문서 섹션 구분자
                                                    "\n【쟁점사항】",     #  쟁점 부분 구분
                                                    "\n【처리경위】",     #  처리 과정 구분
                                                    "\n【처리결과】",     #  결과 부분 구분
                                                    "\n■", "\n\n", "\n", ".", " ", ""
                                                    ])

chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

print("==> 3. 벡터화 → 임베딩으로 변환")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

print("===> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)
print(" Retriever 설정 완료")

print("===> 6. 생성 → LLM으로 답변 생성")
llm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)

# 한국어 최적화 프롬프트
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["retrieveDocuments", "question"]
)
print(" 프롬프트 설정 완료")

# ===================================
# 7. QA 체인 생성
# ===================================
print("\n ===> 7.  QA 체인 생성...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=retriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": prompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)
print("  RAG 파이프라인 구축 완료!")

# ===================================
# 8. 테스트 질문들
# ===================================
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)

# ===================================
# 9. 질문 및 답변 실행
# ===================================
for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)


==> 1. 문서 로딩 → PDF 읽기...
  총 109페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  104개 청크 생성 완료
  평균 청크 길이: 722자
==> 3. 벡터화 → 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장
 FAISS 벡터스토어 생성 완료 (104개 벡터)
===> 5. 검색 → 질문과 유사한 문서 찾기
 Retriever 설정 완료
===> 6. 생성 → LLM으로 답변 생성
 프롬프트 설정 완료

 ===> 7.  QA 체인 생성...
  RAG 파이프라인 구축 완료!

 RAG 시스템 테스트

【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다.

1. **사례 기반 답변**:
   - 제시된 사례 중 시스템 오류로 인한 아이템 소실에 대한 사례(사례 1-가-1, 2006 및 2009)가 있습니다. 이 사례들에서는 시스템 오류로 인한 아이템 소실에 대해 게임사가 복구를 거부한 경우가 있었습니다. 특히, 2006년 사례에서는 계정 명의자가 아닌 경우 복구가 불가하다는 점이 강조되었습니다.

2. **법적 근거 제시**:
   - 일반적으로 온라인 게임의 이용약관은 계약의 일종으로, 게임사는 약관에 명시된 내용을 근거로 복구 여부를 결정합니다. 따라서, 이용약관을 검토하여 시스템 오류로 인한 복구에 대한 조항이 있는지 확인하는 것이 중요합니다.

3. **판례 참조**:
   - 2009년 사례에서는 시스템 오류로 인한 손실이 발생했으나, 다른 사용자들에게 유사한 불만이 접수되지 않아 시스템 오류로 보기 어렵다는 이유로 복구가 거부되었습니다. 이는 시스템 오류의 